In [1]:
%matplotlib inline
import sys
sys.path.insert(0, 'src')

from utils import source, handlers
from structs import SweepLine as SL
from answers import *
from vis_utils import *
from common import *

hround 23   10
hround 45   10
(2; 4; 1)
new point  (41; 41; 10)
new point  (61; 1; 10)
new point  (21; 21; 10)
new point  (81; 11; 10)
new point  (2; 0; 1)
new point  (61; 40; 10)
new point  (21; -1; 10)
new point  (60; 39; 10)
new point  (2; 1; 1)
new point  (3; 0; 1)


# Snap rounding

## Задача
Зачастую, если мы имеем дело с некоторым набором отрезков, он задан с произвольной точностью. Но для того, чтобы с этими отрезками можно было работать, необходимо каким-то образом привести представление к выбранной конечной точности. Положим величину этой точности равной $2^p$. Тогда можно сказать, что мы хотим отмасштабировать пространство сеткой со стороной ячейки, равной $2^p$, и притянуть заданные концы отрезков к этой сетке.

**Snap rounding** это способ построения набора ломаных, заданных с конечной точностью, на базе множества отрезков, заданного с произвольной точностью. 

Чтобы можно было говорить о соответствии между полученными ломаными и отрезками, необходимо наложить на результат такого приближения условия корректности, но для начала введем базовые понятия.

### Базовые определения и обозначения
Исходя из постановки задачи, мы имеем некоторую величину точности $2^p$, которую хотим получить. Разобьем плоскость на квадратные ячейки со стороной, равной $2^p$. Такие ячейки будем называть **пикселями**.

**Критическая точка** — точка, которая является концом какого-либо отрезка или пересечением двух отрезков.

**Горячий пиксель** — пиксель, содержащий внутри критическую точку.

**Излом** — точка на отрезке, которая будет являться вершиной приближающей ломаной.

Отрезки будем обозначать латинскими буквами, соответствующие им ломаные — созвучными греческими.

<img src="img/sr1.png" style="width: 60%">
Здесь слева изображено исходное расположение отрезков, справа — результат snap rounding'a. Серым выделены горячие пиксели.

### Условия корректности
1. Все вершины ломаных лежат в центрах пикселей.
2. Результирующие ломаные не сильно отстоят от порождающих отрезков. А именно, ломаная $\sigma$, приближающая отрезок $s$, лежит в пределах суммы Минковского $s$ и пикселя, содержащего конец $s$.
3. Отрезки ломаных не пересекаются кроме как в концах.
4. Топологическая корректность: если в исходных данных какая-то точка была справа от отрезка $s$, то после выполнения snap rounding'а она не окажется слева от ломаной $\sigma$ (но может ей принадлежать).

## Преобразование
Найдем все горячие пиксели и рассмотрим два случая:
* Если конец отрезка попадает в горячий пиксель, переместим его в центр этого пикселя.
* Если отрезок проходит через горячий пиксель, изменим его таким образом, чтобы он проходил через центр этого пикселя.
<img src="img/sr_rules1.png" style="width: 30%; float: left;"> <img src="img/sr_rules2.png" style="width: 30%; float: left;"> <p style="clear: both;">

### Корректность
Докажем, что такое преобразование удовлетворяет введенным выше условиям корректности.

Из построения очевидно, что все вершины ломаных лежат в центре пикселей.

**Лемма** *После выполнения snap rounding'а ломаная $\sigma$, соответствующая отрезку $s$, лежит внутри суммы Минковского $s$ и пикселя с центром в конце $s$.*

$\triangleright$
<div style="padding-left:40px">Упорядочим все горячие пиксели, пересекаемые отрезком $s$, по времени прохождения через них. Рассмотрим два соседних горячих пикселя $p_1$ и $p_2$ и фрагмент отрезка $s$, находящийся внутри этих пикселей и между ними. Поскольку пиксель обладает свойством центральной симмтерии, сумма Минковского $s$ и пикселя содержит центры $p_1$ и $p_2$. Такая сумма Минковского, очевидно, выпукла, а значит содержит в себе и звено соответвующей ломаной $\sigma$, соединяющее эти два центра.</div>
$\triangleleft$

<img src="img/fragmentation.png" style="width: 20%; float: right">
Для доказательства топологической корректности рассмотрим непрерывное преобразование $D$, переводящее исходные отрезки в ломаные, полученные в результате snap rounding'а.

Перед началом преобразования разобьем каждый отрезок на фрагменты, введя точки излома в тех местах, где отрезок пересекается с границами горячего пикселя. В случае, если отрезок пересекает общую границу двух горячих пикселей, в месте пересечения вводим две точки излома и фрагмент нулевой длины, их соединяющий.

Деформация $D$ проходит в два шага. 
На первом шаге все горячие пиксели одновременно и равномерно сжимаются по $x$-координате к вертикальной прямой, идущей через центр до тех пор, пока в момент времени $t=1$ все горячие пиксели не схлопнутся в вертикальные отрезки. Все точки излома на границах горячих пикселей следуют за движением своих пикселей. Каждый отрезок, таким образом, непрерывно деформируется в ломаную. В момент времени $t=1$ каждый горячий пиксель превратится в горячую "палку" с расположенными на ней точками излома.
На втором шаге деформации, пусть от $t=1$ до $t=2$, горячие "палки" сжимаются по вертикали одновременно и равномерно к своему центру. Аналогично первому шагу, звенья ломаных следуют за соответствующими точками излома. Нетрудно показать, что в момент времени $t=2$ полученные таким образом ломаные соответствуют описанию ломаных snap rounding'а. 

Заметьте, что расставленные перед началом $D$ точки излома разбивают каждый отрезок на фрагменты двух типов: **внутренние** — те, которые лежат внутри горячих пикселей и **внешние** — те, которые соединяют точки излома на границах разных горячих пикселей. В конце деформации $D$ внутренние фрагменты схлопываются до центра пикселя и остаются только внешние. Ни одна точка не пересекает никакой фрагмент в ходе этой деформации. В частности, вершина ломаной никогда не может пересечь другую ломаную и ориентация элементарного треугольника, сформированного тремя ломаными никогда не инвертируется.

**Теорема** *В ходе деформации $D$ никакая точка излома не переходит через отрезок.*

$\triangleright$
<div style="padding-left:40px">Во-первых, заметим, что для внутренних фрагментов условие теоремы, очевидно, верно. Они все находятся на участке плоскости, на котором происходит однородное преобразование вдоль осей. 

Сформулируем важный инвариант: никакой внешний фрагмент никогда не соприкасается и не заходит внутрь горячего пикселя (не считая концов этого фрагмента). Рассмотрим, например, первый шаг алгоритма (деформиацию по $x$). Чтобы внешний фрагмент попал внутрь горячего пикселя, он должен пройти через один из углов этого пикселя. Более того, в процессе этого прохождения, внешний фрагмент и угол должны двигаться в одном направлении по $x$ (оба вправо или оба влево), поскольку пиксель сжимается. Но скорость углов пикселя по $x$ максимальна по величине среди всех точек на границе пикселя. Вдобавок к этому, точка на фрагменте имеет $x$ составляющую скорости, являющуюся выпуклой комбинацией скоростей его концов, которые будучи точками излома, следуют за границами горячего пикселя. 

Аналогичные рассуждения проводятся и для деформации по $y$.

Таким образом, фрагмент никогда не может пересечь деформирующийся горячий пиксель.</div>
$\triangleleft$

В качестве следствия из вышесказанного, можно вывести некоторые свойства топологического соответствия между исходными сегментами и полученными в результате snap rounding'а ломаными.

**Утверждение**
*Для ломаных, полученных в результате деформации $D$ выполнено следующее:*
* Никакие отрезки не пересекаются кроме как в концах
* Взаимное расположение фрагментов, оканчивающихся в одном пикселе, соответствует расположению исходных отрезков, пересекающих этот пиксель.
* Если отрезок $r$ пересекается с отрезком $s$ и затем с отрезком $t$, то ломаная $\rho$ не может пересекать ломаную $\tau$ раньше, чем ломаную $\sigma$.
* Если вертикальная прямая $l$, проходящая через центры пикселей, пересекает отрезок $s$ и затем отрезок $t$, то прямая $l$ не может пересекать ломаную $\tau$ раньше, чем ломаную $\sigma$.

$\triangleright$
<div style="padding-left:40px">
* В начале деформации никакие внешние фрагменты не пересекаюся (кроме как в концах), и это свойство сохраняется в ходе $D$, что показано в теореме. Более того, все внутренние фрагменты схлопываются в точки к окончанию $D$.
* Границы горячих пикселей деформируются непрерывно, таким образом, в ходе $D$ сохраняется порядок точек излома на границах.
* Отрезок $r$ и соответствующая ему ломаная $\rho$ проходят через горячие пиксели в одном порядке. В частности, это применимо к горячим пикселям, порожденным пересечениями $r$ с $s$ и $t$. В случае, если эти пересечения лежат в одном горячем пикселе, после snap rounding'а они будут совпадать.
* Чтобы такая инверсия произошла, отрезки $s$ и $t$ должны пересекаться, а snap rounding должен перенести пересечение вдоль вертикальной прямой через центры пикселей, чего, очевидно, произойти не может.</div>
$\triangleleft$

## Алгоритм
После того, как мы обсудили теоретические аспекты snap rounding'а, возникает вопрос, как же эффективно его реализовать на практике. Самым очевидным ответом на этот вопрос будет следующий алгоритм: вычислим все точки пересечения отрезков, построим множество горячих пикселей и произведем деформацию $D$. 

Далее будет описан более эффективный алгоритм, не требующий предподсчета всех точек пересечений отрезков.

## Goodrich Hershberger
Приводимый алгоритм основан на принципе заметающей прямой.

Основная идея заключается в том, что когда в пикселе обнаруживается критическая точка, мы удаляем все фрагменты отрезков, находящиеся в этом, как теперь известно, горячем, пикселе. Одновременно с этим, мы добавляем верхнюю и нижнюю границы пикселя в статус заметающей прямой как горизонтальные отрезки. Таким образом, когда нам становится про какой-то пиксель известно, что он горячий, мы удаляем все пересечения, происходящие внутри пикселя, а так же создаем события пересечения между отрезками и его границами.

Представленный алгоритм использует три структуры данных:
1. Заметающая прямая Бентли-Оттмана (`line`), которая хранит текущую позицию по $x$ (`xpos`), статус (`status`) и priority-queue событий (`events`).
2. С каждым обнаруженным (содержащим критическую точку, расположенную левее `xpos`) горячим пикселем хранится два отсортированных по $x$ списка отрезков, пересекающих верхнюю (`pix.upper`) и нижнюю (`pix.lower`) границы этого пикселя слева от `xpos`.
3. Отсортированный по $y$ список (`current`) обнаруженных горячих пикселей, пересекающихся с $x=$`xpos`.

Алгоритм обрабатывает пять различных видов событий:
1. Концы отрезков (правые концы идут раньше левых при равном $x$).
2. Пересечения отрезков.
3. Пересечение отрезка с границами пикселя.
4. Повторное вхождение отрезка (происходит на выходе из горячего пикселя).
5. Правые концы границ горячих пикселей (события этого типа обрабатываются раньше, чем 4 при равном $x$).

Перед началом работы инициализируем заметающую прямую событиями типа 1 и 2. События типа 3, 4 и 5 пока не существуют. 

In [2]:
# Пока что псевдокод из статьи

# 1
def seg_endpoint(point, segment):
    pix = Pixel(point)
    if pix not in Hcur:
        heat(pix)
    if "point is the left end of segment":
        pix.segs.insert(segment)
        if "segment intersects pix to the right of point at pp":
            sl.events.insert(seg_reinsertion(pp, pix, segment))
    else:
        sl.status.remove(segment)

# 2
def segseg_intersection(point):
    heat(Pixel(point))

# 3
def segpix_intersection(point, pix, segment):
    if "point is on the top of pix":
        pix.toplist.append(segment)
    else:
        pix.botlist.append(segment)
    pix.segs.insert(segment)
    sl.status.remove(segment) # as if segment ended at point
    if "segment intersects pix to the right of point at pp":
        sl.events.insert(seg_reinsertion(pp, pix, segment))

# 4
def seg_reinsertion(point, pix, segment):
    sl.status.insert(segment)
    if "p is on the top of pix":
        pix.toplist.append(segment)
    elif "p is on the bottom of pix":
        pix.botlist.append(segment)
    ppix = "pixel, adjacent to pix with point on their common boundary"
    if ppix in Hcur:
        segpix_intersection(point, ppix, segment)

# 5
def pix_end(pix, point):
    "remove pix boundaries from sl"
    H.extend(Hcur)
    Hcur = []

Чтобы понять, как устроена функция `heat(pixel)`, необходимо доказать следующую лемму.

**Лемма** *Пусть имеется пиксель `pix` $\notin$ `current`, пересекающий прямую $x = $ `xpos`. Пусть `pixup` и `pixdown` — пиксели из `Hcur` над и под `pix` соответственно, если таковые существуют. Пусть `rect` — прямоугольник (возможно, бесконечный), ограниченный `pixup`, `pixdown`, `xpos` и координатой по $x$ левой границы `pix`. Тогда все отрезки, проходящие через `rect`, принадлежат `pixup.lower`, `pixdown.upper` или статусу заметающей прямой. В каждом из этих списков отрезки, пересекающие `pix`, идут по порядку и найти их можно за $O(\log{n} + k)$, где $k$ — количество таких отрезков*

$\triangleright$
<div style="padding-left:40px">
По определению `current`, пересечения и концы никаких отрезков не лежат в `rect`. Тогда любой отрезок, проходящий через `pix`, проходит также и через `rect` и должен пересечь его верхнюю (и попасть в `pixup.lower`), нижнюю (`pixdown.upper`) или правую (статус заметающей прямой) границы. Если два отрезка $s$ и $s'$, лежащих в списке для одной из этих границ (обозначим эту границу за $e$), пересекают `pix`, то внутри `pix` можно провести отрезок $e'$, их соединяющий. Отрезки $e$, $e'$, $s$ и $s'$ образуют четырехугольник. Любой отрезок, лежащий между $s$ и $s'$ в списке для $e$, пересекает границу $e$ и заходит в четырехугольник. При этом, отрезок не пересекает $s$ и $s'$, значит он должен пересечь $e'$ и, соответственно, `pix`. Таким образом, все отрезки, пересекающие `pix`, лежат в списках подряд. Найти подпоследовательность таких отрезков в каждом из трех списков мы можем с помощью бинпоиска: для каждого отрезка $s$ в списке, не пересекающего `pix`, можно за конечное время вычислить, лежат ли искомые отрезки до или после $s$.
</div>
$\triangleleft$

Теперь читателю предлагается реализовать `heat(pix)`.

In [3]:
# Пока что псевдокод из статьи

def heat(pix):
    Hcur.insert(pix)
    segs = "find segments that intersect pix left of xpos, using Lemma"
    for u in segs:
        pix.segs.insert(u)
        if "u intersects pix right of xpos":
            sl.status.remove(u)
            if "u intersects the boundary of pix right of xpos at point p":
                sl.events.insert(seg_reinsertion(p, pix, u))
    "build pix.toplist and pix.botlist by sorting pix.segs"
    "insert horizontal segments into SL at the top and bottom boundaries of pix, \
        extending from xpos to the right end of pix"
    "for each pixel boundary bdy inserted, with right endpoint p, \
        schedule an event pix_end(bdy, p)"
            

**Теорема** *Имея множество $S$, состоящее из $n$ отрезков на плоскости и равномерную сетку пикселей $G$, можно построить snap rounding отрезков из $S$ по $G$ за время $O(n\log{n}+\sum_{h\in H}{|h|\log{n}})$, где $H$ — множество горячих пикселей, а $|h|$ — количество отрезков, пересекающих горячий пиксель $h\in H$.*

$\triangleright$
<div style="padding-left:40px">
Пиксель является горячим тогда и только тогда, когда он содержит конец или пересечение отрезков. Приведенный алгоритм обрабатывает такие ситуации в (1) и (2) случаях. Отрезки удаляются из статуса заметающей прямой только внутри тех пикселей, про которые уже известно, что они горячие, так что все горячие пиксели будут обнаружены.
    
Пересечения отрезков с границами горячих пикселей обрабатываются в случае (3) и внутри функции `heat(pix)` (которую вызывают из (1) и (2)). Таким образом будут обнаружены все пересечения, поскольку любой отрезок, пересекающий пиксель `pix`, при этом проходящий его насквозь, должен пересечь верхнюю или нижнюю границы пикселя, или все вертикальные отрезки, соединяющие верхнюю и нижнюю границы. Функция `heat(pix)` находит все пересечения и верхней и нижней границыми пикселя слева от `xpos`, а также все пересечения с вертикальным отрезком, проходящим черех `pix` в $x=$ `xpos`. Случай (3) находит все пересечения с верхней и нижней границами справа от `xpos`, при котором был вызван `heat(pix)`.

Пусть $m=\sum_{h\in H}{|h|}$. Нетрудно убедиться, что время работы `heat(pix)` равно $O(k\log{n})$, где $k$ — количество отрезков, добавленных в `pix.segs`. На все вызовы `heat()` затрачивается $O(m\log{n})$. Остальная часть алгоритма требует только $O(\log{n})$ времени на обработку каждого события. Количество событий (1) типа равно $2n=O(m)$; количество событий (2) и (5) типов равно $O(h)$, где $h$ — количество горячих пикселей, что равно $O(m)$; и количество отрезков (3) и (4) типов равно $O(m)$.

Когда все пересечения отрезков с горячими пикселями обнаружены, порядок, в котором отрезки пересекают границы пикселя можно получить за $O(m\log{n})$, отсортировав их. (Порядок уже известен для пересечений с верхней и нижней границами каждого горячего пикселя.) Имея эту информацию, можно легко построить snap rounding для $S$.
</div>
$\triangleleft$

In [4]:
def segment_endpoint(point, segment):
    segment_endpoint_answer(point, segment)

def segseg_intersection(point):
    segseg_intersection_answer(point)

def segpix_intersection(point, segment, pixel):
    segpix_intersection_answer(point, segment, pixel)

def segment_reinsertion(point, segment, pixel):
    segment_reinsertion_answer(point, segment, pixel)

def pixel_end(point, pixel):
    pixel_end_answer(point, pixel)
    
def heat(pixel):
    heat_answer(pixel)

handlers[SL.Event.Type.SEG_END] = segment_endpoint
handlers[SL.Event.Type.SEG_START] = segment_endpoint
handlers[SL.Event.Type.SEG_SEG] = segseg_intersection
handlers[SL.Event.Type.SEG_PIX] = segpix_intersection
handlers[SL.Event.Type.SEG_REINSERT] = segment_reinsertion
handlers[SL.Event.Type.PIX_END] = pixel_end

In [5]:
def break_into_pieces(folder = '.pieces'):
    dump = create_dump_func(folder, visual_dump_pieces, line.status, segments, hot, current)
    dump()
    while line.events:
        e = line.pop()
        line.xpos = e.point.x / e.point.z
        print("!", e.etype, " ", e.point)
        e.handle()
        dump(line.xpos)
    dump()
    
break_into_pieces()  #int is 2^63 - 1 so fix it

xpos  None
! Type.SEG_START   (2; 0; 1)
hround 2   1
hround 0   1
rounded  (2; 0; 1)
nice
hhhhhhhheating  (2; 0; 1)
inserting pixel
(25; 5; 10)
(25; -5; 10)
____________
----- ((15; 5; 10), (25; 5; 10))
((15; 5; 10), (25; 5; 10))
--endstatus--
Type.SEG_START   (2; 1; 1)
Type.SEG_START   (21; 21; 10)
Type.SEG_START   (21; -1; 10)
Type.SEG_END   (60; 39; 10)
Type.SEG_END   (3; 0; 1)
Type.SEG_END   (61; 40; 10)
Type.SEG_START   (41; 41; 10)
Type.SEG_END   (81; 11; 10)
Type.SEG_END   (61; 1; 10)
--endevents--
new point  (5; 1; 2)
----- ((15; -5; 10), (25; -5; 10))
((15; -5; 10), (25; -5; 10))
((15; 5; 10), (25; 5; 10))
--endstatus--
Type.SEG_START   (2; 1; 1)
Type.SEG_START   (21; 21; 10)
Type.SEG_START   (21; -1; 10)
Type.SEG_END   (60; 39; 10)
Type.PIX_END   (5; 1; 2)
Type.SEG_END   (61; 40; 10)
Type.SEG_START   (41; 41; 10)
Type.SEG_END   (81; 11; 10)
Type.SEG_END   (61; 1; 10)
Type.SEG_END   (3; 0; 1)
--endevents--
intersection of  ((15; -5; 10), (25; -5; 10))  and  ((15; 5; 10), (25; 

start
(2; 1; 1)
(5; 1; 2)
end


xpos  2.5
! Type.SEG_REINSERT   (205; 40; 82)
inserting  ((2; 0; 1), (61; 40; 10))
----- ((2; 0; 1), (61; 40; 10))
((2; 0; 1), (61; 40; 10))
((2; 1; 1), (3; 0; 1))
--endstatus--
Type.SEG_REINSERT   (975; 121; 390)
Type.SEG_REINSERT   (75; 61; 30)
Type.SEG_START   (41; 41; 10)
Type.SEG_END   (60; 39; 10)
Type.SEG_END   (3; 0; 1)
Type.SEG_END   (61; 40; 10)
Type.SEG_END   (81; 11; 10)
Type.SEG_END   (61; 1; 10)
--endevents--
intersection of  ((2; 0; 1), (61; 40; 10))  and  ((2; 1; 1), (3; 0; 1))  is  (-203; -40; -81)
new point  (203; 40; 81)
hround 25   10
hround 0   10
xpos 

start
(2; 0; 1)
(205; 40; 82)
end


 2.5
! Type.SEG_REINSERT   (975; 121; 390)
inserting  ((21; -1; 10), (60; 39; 10))
----- ((21; -1; 10), (60; 39; 10))
((21; -1; 10), (60; 39; 10))
((2; 0; 1), (61; 40; 10))
((2; 1; 1), (3; 0; 1))
--endstatus--
Type.SEG_REINSERT   (75; 61; 30)
Type.SEG_SEG   (203; 40; 81)
Type.SEG_START   (41; 41; 10)
Type.SEG_END   (60; 39; 10)
Type.SEG_END   (3; 0; 1)
Type.SEG_END   (61; 40; 10)
Type.SEG_END   (81; 11; 10)
Type.SEG_END   (61; 1; 10)
--endevents--
intersection of  ((21; -1; 10), (60; 39; 10))  and  ((2; 0; 1), (61; 40; 10))  is  None
hround 25   10
hround 0   10


...

xpos  5.5
! Type.SEG_END   (60; 39; 10)
hround 60   10
hround 39   10
rounded  (6; 4; 1)
nice
hhhhhhhheating  (6; 4; 1)
inserting pixel
(65; 45; 10)
(65; 35; 10)
____________
----- ((55; 45; 10), (65; 45; 10))
((41; 41; 10), (61; 1; 10))
((21; 21; 10), (81; 11; 10))
((21; -1; 10), (60; 39; 10))
((2; 0; 1), (61; 40; 10))
((55; 45; 10), (65; 45; 10))
--endstatus--
Type.SEG_END   (61; 1; 10)
Type.SEG_END   (61; 40; 10)
Type.SEG_END   (81; 11; 10)
--endevents--
intersection of  ((55; 45; 10), (65; 45; 10))  and  ((2; 0; 1), (61; 40; 10))  is  None
new point  (13; 9; 2)
----- ((55; 35; 10), (65; 35; 10))
((41; 41; 10), (61; 1; 10))
((21; 21; 10), (81; 11; 10))
((55; 35; 10), (65; 35; 10))
((21; -1; 10), (60; 39; 10))
((2; 0; 1), (61; 40; 10))
((55; 45; 10), (65; 45; 10))
--endstatus--
Type.SEG_END   (61; 1; 10)
Type.SEG_END   (61; 40; 10)
Type.SEG_END   (81; 11; 10)
Type.PIX_END   (13; 9; 2)
--endevents--
intersection of  ((55; 35; 10), (65; 35; 10))  and  ((21; 21; 10), (81; 11; 10))  is  

No handles with labels found to put in legend.


! Type.PIX_END   (17; 1; 2)
xpos  8.5


No handles with labels found to put in legend.


xpos  None


In [6]:
from IPython.display import display
display(SlideShower('.pieces'))